<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Scraping/main_rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

def get_ranking_from_call(url_business, lang, sort, query):
    headers = [{"name": "Accept", "value": "*/*"}, {"name": "Accept-Encoding", "value": "gzip, deflate, br"},
               {"name": "Accept-Language", "value": "it-IT,it;q=0.8,en-US;q=0.5,en;q=0.3"},
               {"name": "Connection", "value": "keep-alive"},
               {"name": "Content-Type", "value": "application/x-www-form-urlencoded; charset=utf-8"}, {"name": "Cookie",
                                                                                                       "value": "qntcst=D; hl=en_US; wdi=1|3C26116D69138F61|0x1.78d019f71a444p+30|a7756ff94751d3a9; _ga=GA1.2.3C26116D69138F61; location=%7B%22city%22%3A+%22New+York%22%2C+%22state%22%3A+%22NY%22%2C+%22country%22%3A+%22US%22%2C+%22latitude%22%3A+40.713%2C+%22longitude%22%3A+-74.0072%2C+%22max_latitude%22%3A+40.8523%2C+%22min_latitude%22%3A+40.5597%2C+%22max_longitude%22%3A+-73.7938%2C+%22min_longitude%22%3A+-74.1948%2C+%22zip%22%3A+%22%22%2C+%22address1%22%3A+%22%22%2C+%22address2%22%3A+%22%22%2C+%22address3%22%3A+null%2C+%22neighborhood%22%3A+null%2C+%22borough%22%3A+null%2C+%22provenance%22%3A+%22YELP_GEOCODING_ENGINE%22%2C+%22display%22%3A+%22New+York%2C+NY%22%2C+%22unformatted%22%3A+%22New+York%2C+NY%2C+US%22%2C+%22accuracy%22%3A+4.0%2C+%22language%22%3A+null%7D; xcj=1|Ptt9P03gfc75x_PBT9zmqCkUuSuyB7PR-wWUBvABNi4; __qca=P0-60561249-1581956668708; G_ENABLED_IDPS=google; __cfduid=db8764ff59d8028a6c2e1b214867927d81583160194; _gid=GA1.2.2014867238.1583835527; bse=05dcd9d5de304ef0b1d9a76fa768b10f; sc=8a1ca0dbc2; pid=505721aa4569e7bb"},
               {"name": "Host", "value": "www.yelp.com"},
               {"name": "Referer", "value": "https://www.yelp.com/biz/noche-de-margaritas-new-york"},
               {"name": "TE", "value": "Trailers"}, {"name": "User-Agent",
                                                     "value": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0"},
               {"name": "X-Requested-By-React", "value": "true"},
               {"name": "X-Requested-With", "value": "XMLHttpRequest"}]
    headers_ok = {}
    for header in headers:
        temp = {
            header['name']: header['value']
        }
        headers_ok.update(temp)

    x = 0
    reviews_list = []
    position = 1
    url = url_business + "/review_feed?rl=" + lang + "&sort_by=" + sort + "&q=" + query


    #creo e apro un file .csv per scrivere
    business_id = url_business.lstrip('https://www.yelp.com/biz/')
    file_parsed = open('%s' % business_id + '_%s' % sort + '.csv', 'a')
    #creo il writer
    writer = csv.writer(file_parsed)
    #creo header del csv
    header = ['user_id','text_review']
    writer.writerow(header)

   
    while 1:
        if x == 0:
            page_load = requests.get(url + '&start=', headers=headers_ok)
        else:
            page_load = requests.get(url + '&start=' + str(x), headers=headers_ok)
        x = x + 20
        reviews = page_load.json()['reviews']
        
        #print(json.dumps(reviews, indent=4, sort_keys=True))
        if not reviews:
            break
        for review in reviews:
            # Qui potete modificare, prendendovi tutti i dati che volete e scegliete anche in che tipo di struttura dati
            # metterli. Io ho usato una semplice lista, ma se dovete prendere tanti dati di tante review vi consiglio un
            # dataframe
            #reviews_list.append((position, review['userId'], review['user']['reviewCount']))
            business_name = review['business']['name']
            reviews_list.append(review['userId'])
            position = position + 1


            #ciclo per estrarre dal file json i valori di id, età, genere ed etnia
            writer.writerow(['%s' % review['userId'],
                             '%s' % review['comment']['text']]) 

    # Upload user reviews on Google Drive
    if sort == 'relevance_desc' :
      folder_id = '1NnLLE1XA71DpfejYitu-sCFKJHnR4b01'
    else :
      folder_id = '1QfcviYBDEty2tBT1OmS-dMchGD4nnyM4'

    drive_file = drive.CreateFile({'title': '%s' % business_name + '_%s' % sort + '.csv', 'parents': [{'id': folder_id}]})
    drive_file.SetContentFile('%s' % business_id + '_%s' % sort + '.csv') # path of local file content
    drive_file.Upload()  # Upload the file.

    return reviews_list

In [8]:
def RBO(l1, l2, p):
    """
        Calculates Ranked Biased Overlap (RBO) score. 
        l1 -- Ranked List 1
        l2 -- Ranked List 2
    """
    if l1 == None: l1 = []
    if l2 == None: l2 = []
    
    sl,ll = sorted([(len(l1), l1),(len(l2),l2)])
    s, S = sl # s = length of smaller list, S = Smaller List
    l, L = ll # l = length of longer list, L = Longer list
    if s == 0: return 0

    # Calculate the overlaps at ranks 1 through l 
    # (the longer of the two lists)
    ss = set([]) # contains elements from the smaller list till depth i
    ls = set([]) # contains elements from the longer list till depth i
    x_d = {0: 0} # overlap holds number of common elements at depth d
    sum1 = 0.0
    for i in range(l):
        x = L[i]
        y = S[i] if i < s else None
        d = i + 1
        
        # if two elements are same then 
        # we don't need to add to either of the set
        if x == y: 
            x_d[d] = x_d[d-1] + 1.0
        # else add items to respective list
        # and calculate overlap
        else: 
            ls.add(x) 
            if y != None: ss.add(y)
            x_d[d] = x_d[d-1] + (1.0 if x in ss else 0.0) + (1.0 if y in ls else 0.0)     
        #calculate average overlap
        sum1 += x_d[d]/d * pow(p, d)
        
    sum2 = 0.0
    for i in range(l-s):
        d = s+i+1
        sum2 += x_d[d]*(d-s)/(d*s)*pow(p,d)

    sum3 = ((x_d[l]-x_d[s])/l+x_d[s]/s)*pow(p,l)

    # Equation 32
    rbo_ext = (1-p)/p*(sum1+sum2)+sum3
    return rbo_ext

In [ ]:
# ESEMPIO DI CHIAMATA AL METODO
def compare_ranking_restaurant(business_id):
  #business_id = '%s' % id
  language = 'en'

  sorting = 'relevance_desc'  # relevance è l'ordinamento di default di Yelp, descending
                              # altri valori possibili: date_desc, date_asc, relevance_asc
  query = ''
  ranking = get_ranking_from_call("https://www.yelp.com/biz/" + business_id, language, sorting, query)

  sorting = 'date_desc'  # relevance è l'ordinamento di default di Yelp, descending
                         # altri valori possibili: date_desc, date_asc, relevance_asc
  ranking2 = get_ranking_from_call("https://www.yelp.com/biz/" + business_id, language, sorting, query)

  #ritorno il risultato di RBO per i due rankings
  #return RBO(ranking,ranking2,p = 0.93)

In [ ]:
#da cambiare:
#id_restaurant
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import csv

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_restaurant = 'e4NQLZynhSmvwl38hC4m-A'
compare_ranking_restaurant(id_restaurant)

In [ ]:
import csv
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Auto-iterate through all files that matches this query
#file_list = drive.ListFile({'q': "'1jBSsZmUAvND8XzeRo7TO4r41f3BKhUn4' in parents and trashed=false"}).GetList()

Restaurant_Info = '1A34NdENmmk2h-7sK0jfzv7FO14ho9l9F'
download = drive.CreateFile({'id': Restaurant_Info})
download.GetContentFile('Restaurant_Info.csv')

Restaurant_RBO = '1yl6Pg6V4iHmScdNykfqwk8Tq6P2TV1hg'
download = drive.CreateFile({'id': Restaurant_RBO})
download.GetContentFile('Restaurant_RBO.csv')

#creo e apro un file .csv per scrivere
restaurant_RBO = open('Restaurant_RBO.csv', 'a')

#creo il writer
writer = csv.writer(restaurant_RBO)

#creo header del csv
#header = ['business_id', 'business_name', 'rbo']
#writer.writerow(header)

#apro il file csv dove ci sono le info ei ristoranti e lo leggo
with open('Restaurant_Info.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for riga in csv_reader:
      #se è l'header vai alla riga successiva
      if(line_count <= 0):
        line_count = line_count + 1
      else:
        if (line_count == 2):
          break
        else:
          #tolgo le parentesi all'inizio e alla fine del business_id
          id = riga[2].lstrip(' (').rstrip(')')
          rbo = compare_ranking_restaurant(id)
          line_count = line_count + 1

#salvo e chiudo il file .csv
restaurant_RBO.close()

download.SetContentFile('Restaurant_RBO.csv')  # path of local file content
download.Upload() 

In [10]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
nome = 'Lindo Michoacan'
relevance = '1Sdu5-9JAmGCE1bJ6HGJ5n1nAQAOUrNpG'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': relevance})
download.GetContentFile('%s' % nome + '_relevance_desv.csv')

dfr = pd.read_csv('%s' % nome + '_relevance_desv.csv')



date = '1A4pGcvX3v-tI-VMXpbWLc2H4PBCm2_Et'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': date})
download.GetContentFile('%s' % nome + '_date_desc.csv')

dfd = pd.read_csv('%s' % nome + '_date_desc.csv')

In [12]:
lista_relevance = []

for i,row in dfr.iterrows():
  lista_relevance.append((row['user_id'],row['text_review']))


lista_date = []

for i,row in dfd.iterrows():
  lista_date.append((row['user_id'],row['text_review']))


print(RBO(lista_date,lista_relevance,0.93))

[('qbeaRjhB8A_6Jjw6YTS-hw', 'It was my brothers birthday and we had been running around for most of the day , by the time we decided to get dinner it was almost 9pm ! We called Lindo Michoacan to ask what time they closed and to let them know we were coming in with a large party of 20 total , they said no problem . <br>When we arrived the lady at the front was really nice &amp; friendly , they already had set up our table and we got seated right away . <br>Our waiter was Javier , he was awesome ! <br>He made us feel welcome and he was so helpful with our orders , there were 2 other guys helping him and they were also very nice . One of the guys was joking around and I loved his energy ! <br>Our food came out and everything was perfect , the handmade tortillas were bomb ! And the kids loved the Fideo soup. <br><br>We will definitely be coming back every time we&#39;re in town and we&#39;ll definitely recommend them when our friends visit Vegas . <br><br>Thanks guys you did an awesome jo